In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from cities.models import *
from scipy import stats
import ipy_table as tbl

run_id = 96
qid = 1053

In [2]:
import spacy
from geotext import GeoText
nlp = spacy.load('en')
doc = nlp(u'London is a big city in the United Kingdom.')
docs = Doc.objects.filter(query=qid,cities__isnull=True)
print(docs.count())

docplaces = 0

spacy = False
geotext = True

for d in docs:
    if spacy:
        doc = nlp(d.content)
        #print(d.content)
        places = [x for x in doc.ents if x.label_=="GPE"]
        if len(places) > 0:
            docplaces+=1
            print("############### \n## SPACY")
            for place in places:
                print(place.label_, place.text)
                try:
                    dbc = City.objects.filter(
                        name__unaccent=place.text
                    ).order_by('-population').first().name_std
                except:
                    dbc = "not found"
                print(dbc)
 
    if geotext:
        gplaces = set(GeoText(d.content).cities)
        if len(gplaces) > 0:
            for place in gplaces:
                try:
                    dbc = City.objects.filter(
                        alt_names__name__unaccent=place
                    ).order_by('-population').first()
                    d.cities.add(dbc)
                    docplaces+=1
                except:
                    dbc = "not found"
        
print(docplaces)

7166
3349


In [51]:
badcities = ['Metro','Most','Sim','Young','University','Green','Much','Mobile','Federal','Along','Of','Laplace']
cities = City.objects.filter(doc__query=qid).exclude(name__in=badcities)
cities = cities.annotate(
    n = Count('doc')
).order_by('-n').values('name','country__name','n','location')

docs = Doc.objects.filter(query=qid,cities__isnull=False).exclude(cities__name__in=badcities)
print(docs.count())


2696


In [52]:
df = pd.DataFrame.from_dict(list(cities))
df.to_csv("../data/place_mentions.csv")
df.head(135)

,country__name,location,n,name
0,United Kingdom,"[-0.12574, 51.50853]",117,London
1,United States,"[-74.00597, 40.71427]",102,New York City
2,China,"[116.39723, 39.9075]",102,Beijing
3,Sweden,"[18.0649, 59.33258]",73,Stockholm
4,China,"[121.45806, 31.22222]",43,Shanghai
5,Australia,"[151.20732, -33.86785]",41,Sydney
6,Canada,"[-79.4163, 43.70011]",39,Toronto
7,Chile,"[-70.64827, -33.45694]",38,Santiago
8,Australia,"[144.96332, -37.814]",38,Melbourne
9,United States,"[-122.67621, 45.52345]",36,Portland


In [50]:
aldocs = Doc.objects.filter(cities__name="Laplace")
for al in aldocs:
    print(al.content)

In this paper, a self-adaptive tolling strategy (SATS) is developed for dynamically and systematically enhancing high-occupancy toll (HOT) lane system operations. This strategy enhances the overall system performance of both the HOT and general purpose (GP) lanes by better utilizing the HOT lane capacity while maintaining high speed and/or high travel-time reliability for HOT lane traffic when GP lanes are congested. To formulate SATS, the Lighthill-Whitham-Richards kinematic wave model is used to characterize HOT lane traffic flow evolution, and the unilateral Laplace transform is used to convert the system representation from the time domain to the frequency domain. Then, an adaptive tolling controller is designed with both the proportional and integral control components. Real-time traffic measurements, including lane occupancy, average speed, and flow rate, are utilized for toll rate calculations. Following a dual-phase control scheme, the appropriate flow rate for HOT lane utiliza